### urth-core-channel Examples

### Install Dependencies

In [ ]:
%%html
<link rel='import' href='urth_components/paper-slider/paper-slider.html'
        is='urth-core-import' package='PolymerElements/paper-slider'>

### Example 1: Default channel

`urth-core-channel` provides a mechanism to store shared data. Property values can be stored with the `set` API and retrieved with the `get` API. If a channel name is not specified, a `default` channel is used

In [ ]:
%%html
<urth-core-channel id='defaultChannel'></urth-core-channel>

In [ ]:
%%javascript
var defaultChannel = document.getElementById('defaultChannel');
defaultChannel.set('foo', 'bar');
var val = defaultChannel.get('foo');
element.append('value of foo is: ' + val);

### Example 2: Multiple instances

Multiple instances of the same channel name share data. Setting a property value on one instance is accessible from the other instance.

In [ ]:
%%html
<urth-core-channel id='aChannelOne' name='a'></urth-core-channel>
<urth-core-channel id='aChannelTwo' name='a'></urth-core-channel>

In [ ]:
%%javascript
var aChannelOne = document.getElementById('aChannelOne');
var aChannelTwo = document.getElementById('aChannelTwo');

aChannelOne.set('color', 'blue');
var color = aChannelTwo.get('color');

aChannelTwo.set('size', 'large');
var size = aChannelOne.get('size');

element.append('color of a is: ' + color + '<br>size of a is: ' + size);

### Example 3: Multiple channels

Uniquely named channels are independent. Setting the value for a property in one channel does not affect the value of the property in other channels.

In [ ]:
%%html
<urth-core-channel id='bChannel' name='b'></urth-core-channel>
<urth-core-channel id='cChannel' name='c'></urth-core-channel>

In [ ]:
%%javascript
var bChannel = document.getElementById('bChannel');
var cChannel = document.getElementById('cChannel');

bChannel.set('type', 'car');
cChannel.set('type', 'van');
var bType = bChannel.get('type');
var cType = cChannel.get('type');

element.append('type of b is: ' + bType + '<br>type of c is: ' + cType);

### Example 4: Registering elements

Elements can be registered with a data channel to be notified of property changes.

In [ ]:
%%html
<urth-core-channel id='dChannel' name='d'></urth-core-channel>
<template is='dom-bind' id='theforce'>
    <div><span>{{user}}</span>, may the force be with you</div>
</template>

<script>
    var dChannel = document.getElementById('dChannel');
    var theforce = document.getElementById('theforce');
    dChannel.register(theforce);
    dChannel.set('user', 'Luke');
</script>

### Example 5: Switch channel

Changing the specified channel, changes the data associated with the element.

In [ ]:
%%html
<urth-core-channel id='eChannel' name='e'></urth-core-channel>
<urth-core-channel id='fChannel' name='f'></urth-core-channel>

In [ ]:
%%javascript
var eChannel = document.getElementById('eChannel');
var fChannel = document.getElementById('fChannel');

eChannel.set('planet', 'earth');
fChannel.set('planet', 'saturn');
element.append('<div>planet before switch: ' + eChannel.get('planet') + '</div>');

eChannel.name = 'f';

element.append('<div>planet after switch: ' + eChannel.get('planet') + '</div>');

### Example 6: Initializing the channel

Channel data can be initialized by nesting `urth-core-channel-item` elements inside of the `urth-core-channel`.

In [ ]:
%%html
<urth-core-channel id='itemChannel' name='initialized'>
    <urth-core-channel-item key='myvar' value='awesome'></urth-core-channel-item>
    <urth-core-channel-item key='isAwesome' value=true></urth-core-channel-item>
    <urth-core-channel-item key='otherStuff' value='{ "json": true, "foo": "bar"}'></urth-core-channel-item>
</urth-core-channel>
<template is='urth-core-bind' channel='initialized'>
    <div>myvar is <span>[[myvar]]</span></div>
    <div>isAwesome is <span>[[isAwesome]]</span></div>
    <div>otherStuff.json is <span>[[otherStuff.json]]</span></div>
</template>

### Example 7: Variable Watching API

Using the following API, you can `watch` a bound variable for changes and register a handler function to run on the when a change occurs.

The handler function should take two arguments, `old` and `new` representing the old value and the new value of the variable after a change occurs.

In the following example, the `handler` will set the value of `{{msg}}` when the watched `x` variable changes. First, execute the following cells:

In [ ]:
%%html
<urth-core-channel id='dChannel' name='d'></urth-core-channel>

In [ ]:
%%html
<template is='urth-core-bind' channel='d'>
    <span>{{x}}</span><paper-slider min="0" max="100" step="1" value="{{x}}"></paper-slider>
    <div>{{msgFromJS}}</div>
    <div>{{msgFromPy}}</div>
</template>

##### JavaScript API

In [ ]:
%%javascript
var dChannel = document.getElementById('dChannel');
var handler = function(name, oldVal, newVal) {
    element.append('<div>Watched ' + name + ' change from ' + oldVal + ' to ' + newVal + '</div>');
    dChannel.set('msgFromJS', 'JS Changed from ' + oldVal + ' to ' + newVal);
};

dChannel.watch('x', handler);

##### Python API

In [ ]:
from urth.widgets.widget_channels import channel

In [ ]:
history = []

def handler(old, new):
    msg = "Hello from the kernel! Old: {} New: {}".format(old, new)
    history.append(msg)
    channel('d').set('msgFromPy', msg)

In [ ]:
channel('d').watch('x', handler)

Now move the slider, which changes `x` and in turn triggers `handler`. You should see the `msg` set in `handler` printed next to the slider.

`handler` also modifies the `history` variable:

In [ ]:
history

### Example 8: Saving, reloading and clearing state

The data in a channel can be saved through the `save` API and a previously loaded state restored through the `reload` API. Data in the channel can be cleared with the `clear` API.

Note that all channels on the page are automatically saved when the notebook is saved and saved channel data automatically restored on reload of the page.

In [ ]:
%%html
<urth-core-channel id='stateChannel' name='state'></urth-core-channel>
<template is='urth-core-bind' channel='state'>
    <paper-slider min='0' max='100' step='1' value='{{x}}'></paper-slider>
    <div>The value of x is <span>[[x]]</span></div>
</template>
<button id='saveButton'>Save</button>
<button id='reloadButton'>Reload</button>
<button id='clearButton'>Clear</button>

<script>
    var stateChannel = document.getElementById('stateChannel');
    var saveButton = document.getElementById('saveButton');
    var reloadButton = document.getElementById('reloadButton');
    var clearButton = document.getElementById('clearButton');

    saveButton.onclick = function() {
        stateChannel.save();
    }
    reloadButton.onclick = function() {
        stateChannel.reload();
    }
    clearButton.onclick = function() {
        stateChannel.clear();
    }
</script>

### Example 9: Debugging the content of a channel

The `urth-core-channel` element can also be used to debug a channel. Just set the `debug` flag.

In [ ]:
%%html
<urth-core-channel debug></urth-core-channel>
<urth-core-channel name="a" debug></urth-core-channel>
<urth-core-channel name="b" debug></urth-core-channel>
<urth-core-channel name="c" debug></urth-core-channel>

Editing data gets reflected right away

In [ ]:
%%html
<template is='urth-core-bind' channel='default'>
    <input value="{{foo::input}}"></input>
</template>

In [ ]:
channel('a').set('aDict', dict(a=1, b=2, c='abc'))
channel('a').set('aList', ["a","b","c"])